<a href="https://colab.research.google.com/github/Matsuura699/Python_RPG/blob/main/Python_RPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **プログラミング1 課題04**

- 氏名: 松浦友哉
- 出席番号: 36
- この課題の取り組み時間: **240分**

In [ ]:
%reset -f
# カンマ区切りの「文字列」でデータが与えられているものとします。
import ipywidgets as widgets
from IPython.display import clear_output

# ▼▼▼ ここから(変更不可) ▼▼▼
item_names_str = 'ひのきのぼう,こんぼう,どうのつるぎ,せいなるナイフ,くさりがま,とげのむち,まどうしのつえ,てつのやり,どくばり,てつのつめ,はがねのつるぎ,てつのオノ,あまぐものつえ,いかづちのつえ,さざなみのつえ,さばきのつえ,おおばさみ,ゆうわくのけん,りりょくのつえ,おおかなづち,はやぶさのけん,ゾンビキラー,ドラゴンキラー,くさなぎのけん,ガイアのつるぎ,ふぶきのつるぎ,いなづまのけん,まじんのオノ,らいじんのけん,もろはのつるぎ,はかいのつるぎ,おうごんのつめ,おうじゃのけん'
item_p_str = '50,50,50,40,40,35,30,30,25,20,20,20,10,10,10,10,10,10,10,10,5,5,5,4,3,3,2,2,2,2,2,2,1'
# ▲▲▲ ここまで(変更不可)▲▲▲

item_have = [0] * 33
slider = widgets.IntSlider(min=1, max=100, value=10) #スライダーを用いてガチャの回数を指定できるようにする
button_pull = widgets.Button(description="ガチャを引く")
button_again = widgets.Button(description="もう一度引く") #ボタンを定義
item_names = item_names_str.split(',')
item_p_txt = item_p_str.split(',')
item_p_int = list(map(float,item_p_txt))

item_have = [0] * len(item_names) #アイテムの所持数を表すリストを作る

def menu():
 print('ガチャ回数')
 display(slider)
 display(button_pull)

 # 持っているアイテムを表示する
 print('持ち物')
 for i in range(len(item_have)):
  target,have =item_names[i],item_have[i]
  if have != 0:
   print(f'{target} ... {have}個')
  else:
   pass
 if item_have.count(0) == len(item_names):
  print('持ち物がありません')
 else:
  pass

def pull(b):
 clear_output()
 number = slider.value
 import random as r
 result = r.choices(item_names,weights=item_p_int,k=number)

 if number == 1:
  print(f'ガチャの結果、以下のアイテムを獲得しました。') #「1連ガチャ」という不自然な日本語が出力されないようにする
 else:
  print(f'{number}連ガチャの結果、以下のアイテムを獲得しました。')

 for i in range(len(item_names)):
  target = item_names[i]
  if target in result:
   counts = result.count(target)
   print(f'{target} ... {counts}個')
   item_have[i] += counts
  else:
   pass

 display(button_again) #ガチャを再試行するボタンを出す

def again(b):
  clear_output()
  menu()          #ボタンを押したとき最初のメニューに戻る

menu()

button_pull.on_click(pull)   #ガチャをボタンを押したときに実行するようにする
button_again.on_click(again) #ガチャを再試行できるようにする

ガチャ回数


IntSlider(value=10, min=1)

Button(description='ガチャを引く', style=ButtonStyle())

持ち物
ひのきのぼう ... 2個
こんぼう ... 6個
どうのつるぎ ... 7個
せいなるナイフ ... 3個
くさりがま ... 3個
とげのむち ... 4個
まどうしのつえ ... 2個
てつのやり ... 3個
どくばり ... 3個
てつのつめ ... 2個
はがねのつるぎ ... 2個
てつのオノ ... 1個
あまぐものつえ ... 1個
いかづちのつえ ... 1個
さざなみのつえ ... 3個
ゆうわくのけん ... 2個
りりょくのつえ ... 3個
おおかなづち ... 1個
ゾンビキラー ... 3個
くさなぎのけん ... 2個
いなづまのけん ... 1個
まじんのオノ ... 1個
らいじんのけん ... 1個
はかいのつるぎ ... 2個
おうごんのつめ ... 1個


In [ ]:
import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(min=0, max=10, value=5)
display('number',slider)

button = widgets.Button(description="実行")

def on_button_click(b):
    print("ボタンが押されました！")

button.on_click(on_button_click)
display(button)

def show_value(x):
    # x がスライダーで選ばれた値
    print(f"選ばれた値: {x}")
    y = x * 2
    print(f"x の2倍は: {y}")

interact(show_value, x=(0, 100))

'number'

IntSlider(value=5, max=10)

Button(description='実行', style=ButtonStyle())

interactive(children=(IntSlider(value=50, description='x'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_value(x)>

In [ ]:
%reset -f
a='AAA'
b='BBB'
c='CCC'
print(a+a+b,c,',DDD') # カンマ区切りで引数を4個を与えている

AAAAAABBB CCC ,DDD


In [ ]:
%reset -f
a='AAA'
b='BBB'
c='CCC'
print(a,sep='---',b,c) # キーワード引数が途中に存在

SyntaxError: positional argument follows keyword argument (ipython-input-7-2347155799.py, line 5)

In [ ]:
%reset -f
items = ['どうのつるぎ','やくそう','どくけしそう','ぬののふく']
for i in range(len(items)):
  print(f'もちもの「{items[i]}」')

もちもの「どうのつるぎ」
もちもの「やくそう」
もちもの「どくけしそう」
もちもの「ぬののふく」


In [ ]:
%reset -f
item_names = ['どうのつるぎ','やくそう','どくけしそう','ぬののふく'] # 初期化(文字列)
item_prices = [100,8,10,10] # 初期化(数値)
for i in range(len(item_names)):
 print(f'{item_names[i]}は、{item_prices[i]}G で購入し、{item_prices[i]*3/4:.0f}G で売却できる。')

どうのつるぎは、100G で購入し、75G で売却できる。
やくそうは、8G で購入し、6G で売却できる。
どくけしそうは、10G で購入し、8G で売却できる。
ぬののふくは、10G で購入し、8G で売却できる。


In [ ]:
%reset -f
item_p_str='0.65 0.30 0.05' # スペース(SP)区切りの文字列
item_p1 = item_p_str.split(' ') # 文字列をSPで分解してリスト化

# 現状で item_p1 の要素は【文字列】 ※計算処理に利用できない
for i in range(len(item_p1)):
  print(item_p1[i]*3)

print('---')

# item_p1 の要素を【数値】に変換
item_p2 = list(map(float,item_p1)) # ★ここがポイント!
for i in range(len(item_p2)):
  print(item_p2[i]*100)

0.650.650.65
0.300.300.30
0.050.050.05
---
65.0
30.0
5.0


In [ ]:
%reset -f
item_names_str = 'ひのきのぼう,こんぼう,どうのつるぎ,せいなるナイフ,くさりがま,とげのむち,まどうしのつえ,てつのやり,どくばり,てつのつめ,はがねのつるぎ,てつのオノ,あまぐものつえ,いかづちのつえ,さざなみのつえ,さばきのつえ,おおばさみ,ゆうわくのけん,りりょくのつえ,おおかなづち,はやぶさのけん,ゾンビキラー,ドラゴンキラー,くさなぎのけん,ガイアのつるぎ,ふぶきのつるぎ,いなづまのけん,まじんのオノ,らいじんのけん,もろはのつるぎ,はかいのつるぎ,おうごんのつめ,おうじゃのけん'
item_p_str = '50,50,50,40,40,35,30,30,25,20,20,20,10,10,10,10,10,10,10,10,5,5,5,4,3,3,2,2,2,2,2,1,1'

item_names = item_names_str.split(',') # 書き換える
item_p = item_p_str.split(',')
item_p2 = list(map(float,item_p)) # 書き換える（複数行でも可）

print(item_names)
print(item_p2)

['ひのきのぼう', 'こんぼう', 'どうのつるぎ', 'せいなるナイフ', 'くさりがま', 'とげのむち', 'まどうしのつえ', 'てつのやり', 'どくばり', 'てつのつめ', 'はがねのつるぎ', 'てつのオノ', 'あまぐものつえ', 'いかづちのつえ', 'さざなみのつえ', 'さばきのつえ', 'おおばさみ', 'ゆうわくのけん', 'りりょくのつえ', 'おおかなづち', 'はやぶさのけん', 'ゾンビキラー', 'ドラゴンキラー', 'くさなぎのけん', 'ガイアのつるぎ', 'ふぶきのつるぎ', 'いなづまのけん', 'まじんのオノ', 'らいじんのけん', 'もろはのつるぎ', 'はかいのつるぎ', 'おうごんのつめ', 'おうじゃのけん']
[50.0, 50.0, 50.0, 40.0, 40.0, 35.0, 30.0, 30.0, 25.0, 20.0, 20.0, 20.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 5.0, 5.0, 5.0, 4.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0]


In [ ]:
%reset -f
item_names = ['どうのつるぎ','やくそう','どくけしそう','ぬののふく']
print(item_names[-8])

IndexError: list index out of range

In [ ]:
%reset -f
item_names_str = 'ひのきのぼう,こんぼう,どうのつるぎ,せいなるナイフ,くさりがま,とげのむち,まどうしのつえ,てつのやり,どくばり,てつのつめ,はがねのつるぎ,てつのオノ,あまぐものつえ,いかづちのつえ,さざなみのつえ,さばきのつえ,おおばさみ,ゆうわくのけん,りりょくのつえ,おおかなづち,はやぶさのけん,ゾンビキラー,ドラゴンキラー,くさなぎのけん,ガイアのつるぎ,ふぶきのつるぎ,いなづまのけん,まじんのオノ,らいじんのけん,もろはのつるぎ,はかいのつるぎ,おうごんのつめ,おうじゃのけん'
item_names = item_names_str.split(',')
item_counts = [0]*len(item_names) # ここを書き換える。33という数値の直接記述は禁止。lenを利用すること。

print(f'item_namesの長さ = {len(item_names)}')
print(f'item_countsの長さ= {len(item_counts)}')
print(item_counts)

item_namesの長さ = 33
item_countsの長さ= 33
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
%reset -f
a = ['60','20','40','-30','90']
print(f'sum(a)= {sum(a)}')
print(f'max(a)= {max(a)}')
print(f'min(a)= {min(a)}')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
%reset -f
items = ['どうのつるぎ','やくそう','どくけしそう','ぬののふく']
print(items) # リストをそのまま引数にする。

['どうのつるぎ', 'やくそう', 'どくけしそう', 'ぬののふく']


In [ ]:
%reset -f
items = ['どうのつるぎ','やくそう','どくけしそう','ぬののふく']
# ここから下を書き換える (リストに要素を追加する処理を追加する)

items.insert(2,'てっかめん')
items.insert(0,'おうじゃのつるぎ')
items.insert(0,'ひかりのたて')

for i in range(len(items)):
  print(items[i])

ひかりのたて
おうじゃのつるぎ
どうのつるぎ
やくそう
てっかめん
どくけしそう
ぬののふく


In [ ]:
%reset -f
items = ['どうのつるぎ','やくそう','どくけしそう','ぬののふく']
target = 'どくけしそう'
if target in items :
  idx = items.index(target) # ★ 要素を検索して、発見したインデックスを返す
  print(f'勇者ヨシヒコの {idx+1}番目 のポケットに {target} が入っている')
else :
  print(f'勇者ヨシヒコは {target} を持っていない')

勇者ヨシヒコの 3番目 のポケットに どくけしそう が入っている


In [ ]:
%reset -f
item_names  = ['やくそう','どくけしそう','まんげつそう','せかいじゅのは']
item_counts = [4,2,0,1] # アイテムの所持数

item = 'q'
print('勇者ヨシヒコは宝箱を開けた、',end='')
print(f'なんと「{item}」を手に入れた。')

# ここに item に応じて item_counts などを更新する処理を記載
if item in item_names:
  idx = item_names.index(item)
  item_counts[idx] += 1
else:
  item_names.append(item)
  item_counts.append(0)
  idx = item_names.index(item)
  item_counts[idx] += 1

print('\n---所持品の確認---')
for i in range(len(item_names)):
  print(f'{item_names[i]}...{item_counts[i]}個')

勇者ヨシヒコは宝箱を開けた、なんと「q」を手に入れた。

---所持品の確認---
やくそう...4個
どくけしそう...2個
まんげつそう...0個
せかいじゅのは...1個
q...1個


In [ ]:
%reset -f
import random as r
rank = ['LR','UR','SSR','SR','R']
p = [1,3,10,20,0]
c = r.choices(rank,weights=p,k=10) # ★k=10を設定
print(c)

['SSR', 'SSR', 'SR', 'SSR', 'SR', 'SR', 'SSR', 'SR', 'SR', 'SR']
